# Applied data science capstone: Data-based decision support to inform relocations

## Introduction
Description of the problem and business case


Relocations, moving to a new place and establishing one's home there due to e.g. change of job, are periods of great changes where several important decisions need to be taken. Among these important decisions, where to live is probably one of the most important ones. In effect, one's home location determines not only how much time will be used for commuting to work/to study or how big one's home will be given an available budget, but also what kind of services (grocery shops, restaurants, schools, cinemas, etc.) will be easily accessible.

In many cases, the decision of where to relocate is taken either quickly or based on limited information, especially when one is relocating far, e.g. to another country. 

This capstone will aim at developing a data-based decision support to help those in the process of relocating. 

To simplify the decision-making process of where to relocate, it is assumed that it depends on the following parameters:

* **Composition of neighbourhood**, this is a subjective criteria that depends on the individual preferences of the person relocating. 
* **Size of the new apartment**, this is a function of available budget and the chosen location (neighbourhood) to relocate. 
* **Commuting time**, this can be modeled as a function of the distance between the chosen location to relocate and the location of the commute (work/study). 

For the purpose of this capstone, the user (i.e. the one relocating) will define its preferences and constraints in terms of:
* **Location (target neighbourhood) he would like the new apartment's location to be similar to**, this can be the current apartment's location if the user finds it is a comfortable neighbourhood. 
* **Available budget**, this will be used to estimate the size of the apartment given a recommended location
* **Location of work/study**, this will be used to estimate commuting time by computing distance between work/study location and the new apartment's location.

The main idea is that users inform i) a neighbourhood location they like, ii) the city where they are rellocating, iii) an available budget and iv) the location of work/study. 

Given the above parameters, the user will be presented with suggested neighbourhoods to relocate. For each suggested neighbourhood, an estimated apartment size and daily commuting time will be calculated. This will provide decision-support to the user, that will then be able to target their apartment search on the recommended neighbourhoods. 

This project could be extended so that not only neighbourhoods, but actually apartments, are proposed to the user relocating. 

In order to suggest neighbourhoods for relocation, the Foursquare location data will be used to characterize the 'target neighbourhood' as well as the different neighbourhoods in the city where to relocate. Then, a clustering algorithm will be used to cluster the set of the neighbourhoods in the new city plus the target neighbourhood. Once similar neighbourhoods to the target one are identified, commuting times and apartment's size will be estimated based on user-provided information. 


## Libraries
Before progressing further, let's import the necessary libraries that will be used. 

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


## Data
Where you describe the data that will be used to solve the problem and the source of the data.


### Neigbourhoods in Barcelona
We will assume that the user wants to relocate to Barcelona.

An overview of Barcelona's districts (each district contains several neighbourhoods) can be seen below: 
<img src="450px-Barcelona_districtes.svg.png" />

The coordinates of the different neighbourhoods in Barcelona will be extracted from <a href="https://en.wikipedia.org/wiki/Districts_of_Barcelona">this Wikipedia page</a>.

These coordinates will be used to explore the different neighbourhoods (plus the target one) in FOURSQUARE.



In [3]:
## Print map of Barcelona and its neighbourhoods
address = 'Barcelona, SPAIN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Barcelona are 41.3828939, 2.1774322.


In [21]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)
map_barcelona

### Represent Barcelona's neighbourhoods
Here we will convert into a Pandas's dataframe the coordinates of Barcelona's neighbourhoods that have been filed into an Excel file once downloaded from the above website.

In [12]:
pd.read_excel('borough_BCN.xlsx', index_col=0)
data2 = pd.read_excel('borough_BCN.xlsx', sheet_name='Sheet1')
data2.head()

,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,La Barceloneta,41.379889,2.189361
1,Ciutat Vella,El Gotic,41.382778,2.176944
2,Ciutat Vella,El Raval,41.379722,2.168056
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.384608,2.182717
4,Eixample,L'Antiga Esquerra de l'Eixample,41.390000,2.155000


In [22]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(data2['Latitude'], data2['Longitude'], data2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

### Sqm price of Barcelona neighbourhoods

The price per square meter of an apprtment in Barcelona will be extracted from <a href="https://www.bcn.cat/estadistica/castella/dades/timm/ipreus/hab2mave/evo/t2mab.htm">https://www.bcn.cat/estadistica/castella/dades/timm/ipreus/hab2mave/evo/t2mab.htm</a>, which is provided by the local council of Barcelona. 
UARE.

## Additional user information 
For the purpose of illustrating this capstone project, the following parameters will be assumed:
* **Location (target neighbourhood) he would like the new apartment's location to be similar to**: a location in Madrid (similar city) will be chosen.
* **New work location**: the user will be working close to 'Sants Station', the main train station in Barcelona. 
* **User's available budget:** The user has an available budget of 300.000 EUR to buy the apartment where to relocte. 

### Location (target neighbourhood)
The target location is defined below:

In [36]:
## Print map of Barcelona and its neighbourhoods
address2 = 'Madrid, Barrio de Salamanca, SPAIN.'
latitude2 = 40.43
longitude2 = -3.677778
print('The target Neighbourhood is', address2, 'Its geograpical coordinates are', latitude2, ',', longitude2,'.' )

The target Neighbourhood is Madrid, Barrio de Salamanca, SPAIN. Its geograpical coordinates are 40.43 , -3.677778 .


In [35]:
# create map of Barcelona using latitude and longitude values
map_madrid = folium.Map(location=[latitude2, longitude2], zoom_start=12)

# add markers to map
folium.CircleMarker(
        [latitude2, longitude2],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3187cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

## Methodology section 
It represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.


## Results 
Section where you discuss the results.


## Discussion 
Section where you discuss any observations you noted and any recommendations you can make based on the results.


## Conclusion 
Section where you conclude the report.

In [3]:
import pandas as pd
import numpy as np